In [72]:
# Copyright (c) 2026 Microsoft Corporation.
# Licensed under the MIT License.

## Config module example

The load_config function provides a comprehensive configuration loading system that automatically discovers and parses YAML/JSON config files into Pydantic models with support for environment variable substitution and .env file loading. It offers flexible features like config overrides, custom parsers for different file formats, and automatically sets the working directory to the config file location for relative path resolution.

In [73]:
from pathlib import Path

import toml
from graphrag_common.config import load_config
from pydantic import BaseModel, Field


class Logging(BaseModel):
    """Test nested model."""

    directory: str = Field(default="output/logs")
    filename: str = Field(default="logs.txt")


class Config(BaseModel):
    """Test configuration model."""

    name: str = Field(description="Name field.")
    logging: Logging = Field(description="Nested model field.")


# Basic - by default:
# - searches for Path.cwd() / settings.[yaml|yml|json]
# - sets the CWD to the directory containing the config file.
#   so if no custom config path is provided than CWD remains unchanged.
# - loads config_directory/.env file
# - parses ${env} in the config file
config = load_config(Config)

# Custom file location
config = load_config(Config, "settings.yaml")

print("Config loaded successfully using basic loading:")

print(f"Name: {config.name}")
print(f"Logging directory: {config.logging.directory}")
print(f"Logging filename: {config.logging.filename}")

Config loaded successfully using basic loading:
Name: my-app
Logging directory: output/logs
Logging filename: app.log


### Using a custom file extension for the configuration file: 

In [74]:
# Using a custom file extension with
# custom config parser (str) -> dict[str, Any]
config = load_config(
    config_initializer=Config,
    config_path="config.toml",
    config_parser=lambda contents: toml.loads(contents),  # Needs toml pypi package
)

print("Config loaded successfully using file extension:")
print(f"Name: {config.name}")
print(f"Logging directory: {config.logging.directory}")
print(f"Logging filename: {config.logging.filename}")

Config loaded successfully using file extension:
Name: my-app-test
Logging directory: output/logs
Logging filename: app.log


### Overriding properties from the config object

In [75]:
# With overrides - provided values override whats in the config file
# Only overrides what is specified - recursively merges settings.
config = load_config(
    config_initializer=Config,
    overrides={"name": "some name", "logging": {"filename": "my_logs.txt"}},
)

print("Config loaded successfully with overrides:")
print(f"Name: {config.name}")
print(f"Logging directory: {config.logging.directory}")
print(f"Logging filename: {config.logging.filename}")

Config loaded successfully with overrides:
Name: some name
Logging directory: output/logs
Logging filename: my_logs.txt


### Resolving relative paths

In [76]:
# By default, sets CWD to directory containing config file
# So custom config paths will change the CWD.
config = load_config(
    config_initializer=Config,
    config_path="config.yaml",
    set_cwd=True,  # default
)

print("Config loaded successfully with overrides:")
print(f"Name: {config.name}")
print(f"Logging directory: {config.logging.directory}")
print(f"Logging filename: {config.logging.filename}")

# And now throughout the codebase resolving relative paths in config
# will resolve relative to the config directory

Path(config.logging.directory)

assert str(Path(config.logging.directory)) == "output/logs"

Config loaded successfully with overrides:
Name: my-app-config
Logging directory: output/logs
Logging filename: logs.txt
